In [1]:
import json
import pandas as pd
import numpy as np
from deepmultilingualpunctuation import PunctuationModel
import pickle
from tqdm import tqdm


2023-06-21 13:24:07.750393: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 13:24:07.923892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 13:24:09.579413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_data(path): 
        merged_df = pd.read_csv(path)
        strings = merged_df['String']
        str_lst = strings.values

        vocab = merged_df['Title'].values
        identifier = merged_df['identifier']
        identifier_vocab = pd.DataFrame({'ID': identifier, 'Vocab': vocab})
        identifier_vocab = identifier_vocab.set_index('Vocab')['ID'].to_dict()
        return merged_df, str_lst, vocab, identifier_vocab, identifier


merged_df, str_lst, vocab, identifier_vocab, identifier = get_data('data/merged_data_for_AI.csv')


In [3]:
model = PunctuationModel()


/root/miniconda3/envs/gpt/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


In [5]:
corrected = []
start_index = 14000
total_iterations = len(merged_df["transcript"])- start_index
with tqdm(total=total_iterations) as pbar:
    for index, t in enumerate(merged_df["transcript"][start_index:]):
        if index % 1000 == 0:
            # Save the corrected list to disk
            with open(f'corrected_transcripts4.pkl', 'wb') as f:
                pickle.dump(corrected, f)
        try:
            result = model.restore_punctuation(t)
            corrected.append(result)
        except Exception as e:
            print(e)
            corrected.append("ERROR: " + repr(e))
        pbar.update(1)

# Save the final corrected list to disk
with open('corrected_final.pkl', 'wb') as f:
    pickle.dump(corrected, f)


  0%|          | 0/57980 [00:00<?, ?it/s]/root/miniconda3/envs/gpt/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 79/57980 [04:42<88:15:25,  5.49s/it] 

In [21]:
with open('corrected_transcripts.pkl', 'rb') as f:
    ctl = pickle.load(f)


In [22]:
len(ctl)

3000

In [26]:
for i in range(3000, 71980, int((71980-3000)/4)):
    print(i)

3000
20245
37490
54735


In [27]:
chunks = [
    merged_df["transcript"][3000:20245],
    merged_df["transcript"][20245:37490],
    merged_df["transcript"][37490:54735],
    merged_df["transcript"][54735:],
]


In [28]:
import torch
from torch.multiprocessing import Pool, Process, set_start_method
from tqdm import tqdm
import pickle

import torch
from torch.multiprocessing import Pool, Process, set_start_method
import pickle
import os

# Function for restoring punctuation
def restore_punctuation(data):
    model = ... # Initialize the model here, if it's a PyTorch model, it could be moved to a GPU.
    model.to('cuda') # Move model to GPU if it's a PyTorch model
    results = []
    for i, (t, index) in enumerate(data):
        result = model.restore_punctuation(t)
        results.append((result, index))
        if i % 1000 == 0:
            print(f'Process {os.getpid()}: processed {i} out of {len(data)} data points')
    # Save the result list to disk
    with open(f'corrected_{index}.pkl', 'wb') as f:
        pickle.dump(results, f)

try:
    set_start_method('spawn')   
except RuntimeError:
    pass





with Pool(processes=4) as pool:
    pool.map(restore_punctuation, chunks)







AttributeError: 'ellipsis' object has no attribute 'to'

In [29]:
import torch
import pickle
import os

# Function for restoring punctuation
def restore_punctuation(data, model):
    results = []
    for i, (t, index) in enumerate(data):
        result = model.restore_punctuation(t)
        results.append((result, index))
        if i % 1000 == 0:
            print(f'Processed {i} out of {len(data)} data points')
    # Save the result list to disk
    with open(f'corrected_{index}.pkl', 'wb') as f:
        pickle.dump(results, f)

def main():
    model = PunctuationModel()
    model.to('cuda') # Move model to GPU if it's a PyTorch model

    # Preparing the data for processing
    data = list(zip(merged_df["transcript"], range(len(merged_df["transcript"]))))

    # Define your batch size here. Adjust this according to your GPU's memory.
    batch_size = 1000

    # Split data into batches and process each batch
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        restore_punctuation(batch, model)

if __name__ == '__main__':
    main()


/anaconda/envs/lj/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


AttributeError: 'PunctuationModel' object has no attribute 'to'

In [3]:
video_embeddings = pd.read_csv('data/sentence_transformer_embedding.csv')
del video_embeddings['Unnamed: 0']
video_embeddings_values = video_embeddings.values
print(video_embeddings_values.shape)

(71980, 768)


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(A, b):
    # Convert A and b to numpy arrays
    A = np.array(A)
    b = np.array(b)

    # Calculate cosine similarity between A and b
    similarity_scores = cosine_similarity(A, b.reshape(1, -1)).flatten()

    # Sort the indices based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[::-1]

    return sorted_indices


In [6]:
def get_top_n_videos(subject,lj_data, video_data_embeddings, video_titles, video_identifier, n = 3):
    subject_dict = lj_data[subject]
    print("Learning journey for ", subject, " is:")
    for i in subject_dict.keys():
        
        print(i)
        tmp_string = subject_dict[i]["name"]["raw_content"]
        t_emb = subject_dict[i]["name"]["embedding"]
        print("to learn ", tmp_string, " watch the following videos:")
        cosine_similarity_sorted = calculate_cosine_similarity(video_data_embeddings,t_emb)
        for index, v in enumerate(cosine_similarity_sorted[:3]):
            title = video_titles[v]
            #print(index)
            #print(title)
            url = "https://www.youtube.com/watch?v=" + video_identifier[v]
            print("Video %i: %s" % (index + 1, title))
            print(url)


In [9]:
study_data_we.keys()

dict_keys(['Betriebswirtschaftslehre', 'Informatik', 'Medizin', 'Maschinenbau', 'Elektrotechnik', 'Psychologie', 'Jura/Rechtswissenschaften', 'Architektur', 'Chemie', 'Biologie', 'Geschichte', 'Soziologie', 'Volkswirtschaftslehre', 'Mathematik', 'Physik', 'Politikwissenschaft', 'Medienwissenschaft', 'Sprachwissenschaft/Linguistik', 'Pädagogik', 'Philosophie', 'Kunstgeschichte', 'Sportwissenschaft', 'Geografie', 'Musikwissenschaft', 'Ethnologie'])

In [11]:
get_top_n_videos("Betriebswirtschaftslehre", study_data_we, video_embeddings_values,merged_df["Title"],merged_df["identifier"])

Learning journey for  Betriebswirtschaftslehre  is:
Grundlagen der Betriebswirtschaftslehre
to learn  Grundlagen der Betriebswirtschaftslehre  watch the following videos:
Video 1: How The Economic Machine Works by Ray Dalio
https://www.youtube.com/watch?v=PHe0bXAIuk0
Video 2: The Theory of Production - Economics A2 Level Unit 3
https://www.youtube.com/watch?v=jph_CTpjNAY
Video 3: 3 Economic Rules of Thumb
https://www.youtube.com/watch?v=X0KOV0o1P30
Marketing
to learn  Marketing  watch the following videos:
Video 1: What is Marketing? Marketing In The Real and Business World - Marketing 101
https://www.youtube.com/watch?v=VzsNqYARnd0
Video 2: How to Develop a Marketing Strategy by Philip Kotler
https://www.youtube.com/watch?v=ghFwpoH71NM
Video 3: Marketing Mix: Promotion Strategy part 2
https://www.youtube.com/watch?v=FT2mJBbU-wE
Rechnungswesen
to learn  Rechnungswesen  watch the following videos:
Video 1: Introduction to Modern Internal Audit (Take Quiz in Description to get 1 CPE)
htt

In [87]:
ll_data_we.keys()

dict_keys(['Barista', 'Gärtner', 'Stricken', 'Basteln', 'Malen', 'Kommunizieren', 'Meditation', 'Campingmanagement', 'Scrum', 'Azure DevOps', 'Projektmanagement', 'Immobilien', 'Teamleitung'])

In [12]:
get_top_n_videos("Azure DevOps", ll_data_we, video_embeddings_values,merged_df["Title"],merged_df["identifier"])

Learning journey for  Azure DevOps  is:
Grundlagen der Azure Cloud
to learn  Grundlagen der Azure Cloud  watch the following videos:
Video 1: Cloud Computing Fundamentals
https://www.youtube.com/watch?v=uroryFU78gM
Video 2: Intro to Google Cloud, a song!
https://www.youtube.com/watch?v=iVFmlgWvJcg
Video 3: Continuous Integration, Continuous Deployment (CI-CD) with Azure DevOps
https://www.youtube.com/watch?v=jRgLSMlp28U
Verwalten von Source Control
to learn  Verwalten von Source Control  watch the following videos:
Video 1: HikCentral Access Control - Access Control Module Setup and Management
https://www.youtube.com/watch?v=nezbYEC0aec
Video 2: Track 2 01 From Workstation to Domain Admin Why Secure Administration Isnt Secure and How to Fix It
https://www.youtube.com/watch?v=Wdbm2_1tn14
Video 3: Control Techniques F600 Pump Drive Energy Features
https://www.youtube.com/watch?v=4LicbciyAbc
Azure Boards
to learn  Azure Boards  watch the following videos:
Video 1: Azure Arc enabled Kubern